In [1]:
# This code analyze mat files generated by "Licking_Optogenetics_LEDCue.m" and save raster plots as pdf.

import os
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt

# set the working directory to the folder containing the .mat files
os.chdir('/Users/jun/Documents/Work/Project/Licking/Analysis_Code/Analsysis_#18,#19_lickometer/')

# loop over all .mat files in the directory
for file in os.listdir():
    if file.endswith(".mat"):
        # load the data from the .mat file
        mat = loadmat(file)
        session_data = mat['SessionData']
        n_trials = session_data['nTrials'][0, 0][0, 0]

        raw_events = session_data['RawEvents'][0, 0]['Trial'][0, 0]

        reward = {
            'BNCHigh': [np.nan] * n_trials,
            'BNCLow': [np.nan] * n_trials
        }

        for i in range(n_trials):
            trial = raw_events[0, i]['Events'][0, 0]

            if 'BNC1High' not in trial.dtype.names or 'BNC1Low' not in trial.dtype.names:
                reward['BNCHigh'][i] = np.nan
                reward['BNCLow'][i] = np.nan
            else:
                reward['BNCHigh'][i] = trial['BNC1High'][0, 0]
                reward['BNCLow'][i] = trial['BNC1Low'][0, 0]


        # Remove empty cells
        reward["BNCHigh"] = [x for x in reward["BNCHigh"] if not np.isnan(x).all()]
        reward["BNCLow"] = [x for x in reward["BNCLow"] if not np.isnan(x).all()]

        # Reward delivery time
        trials = session_data["RawEvents"][0, 0]["Trial"]

        # Extract "States" for each trial
        states_list = [trial.item()[0, 0]["States"][0, 0] for trial in np.nditer(trials, flags=['refs_ok'])]

        # Extract "Drinking" states for each trial
        drinking_list = [states['Drinking'][0] for states in states_list]


        num_trials = len(reward["BNCHigh"])

        # Create a new figure with a specific width and height
        fig = plt.figure(figsize=(12, 8))  # Adjust the width value (6) to make the figure less wide

        for trial in range(num_trials):
            licks = reward["BNCHigh"][trial]
            for lick_arr in licks:
                for lick in lick_arr:
                    ylicks = np.array([trial, trial + 0.2, np.nan])
                    xlicks = np.array([lick, lick, np.nan])
                    plt.plot(xlicks, ylicks, color='SteelBlue', linewidth=2)

        y_height = num_trials + 1
        #plt.ylim([1, y_height])
        plt.ylim([1, 71]) # limit ylim to be consistent across days
        plt.xlim([0, 20])
        plt.ylabel("Trial")
        plt.xlabel("Time (s)")

        reward_time = np.round(drinking_list[0][0][0])
        x_points = [reward_time[0], reward_time[0], reward_time[0] + 1, reward_time[0] + 1]
        y_points = [0, y_height, y_height, 0]
        color = [0, 0.4470, 0.7410]

        cue = plt.fill(x_points, y_points, color, linestyle="none")
        cue[0].set_alpha(0.1)

        plt.axvline(reward_time[0], linestyle=":", color="r", linewidth=1.5)

        # save the plot as a PDF file using the original .mat file name
        filename = os.path.splitext(file)[0] + '.pdf'
        plt.savefig(filename, dpi=1200)

        plt.close()  # close the figure to free memory
